 #### Model Building Steps
    
    1. Understanding the Bussiness problem
    2. Loading and understanding the Data
    3. Exploratory Data Analysis(EDA)
        a.Descriptive Statistics
        b.Data Visualization
    4. Data PreProcessing
        a.Identifiying Num and Cat data and converting them to appropriate dtype
        b.Dectecting the outliers and removing.
        c.Data Wrangling or Data cleaning(Imputation)
                (only if more than 25% of the data is missing, or else remove those rows)
    5. Model Building
        a.Assining X and y status to attributes and target
        b.creating dummies (or) perform label, onehotencoding for categorical attribute.    
        c.Train-Test split.
        d.Scaling(if required).
        e.Training the Model.
        f.Prediction on test data.
    6. Evaluate the model with the appropriate Metrics.
    7. Hyper parmeter tuning if required and updating the model.
    8. Inference about the model

## 1.UNDERSTANDING THE BUSSINESS PROBLEM

****************************************************

## 2.COLLECTION AND LOADING THE DATA  
    

In [ ]:
######Importing date from data directly as date format and making as index(for time series)

data = pd.read_csv("TimeSeries.csv",parse_dates=['Date'],index_col=['Date'])

****************************************************

## 3.EXPLORATORY DATA ANALYSIS 
		

###### Descriptive statistics – Mean, median, Standard deviation, etc.

In [ ]:
data.info()
data.describe()

###### Data visualization – Scatter plot, corelation heatmap, box plot, Count plot of target column(if categorical),etc.

In [ ]:
# Histogram
data.plot(kind='hist',subplots=True,layout=(3,3),sharex=False,figsize=(20,20))

In [ ]:
# Boxplot
data.plot(kind='box',subplots=True,layout=(3,3),sharex=False,figsize=(20,20))

In [ ]:
# pairplot
sns.pairplot(data)

In [ ]:
# Visualizing Correlation amongst numeric attributes using heatmap
cmap = sns.diverging_palette(260,-10,s=50, l=75, n=6, as_cmap=True)
plt.subplots(figsize=(18,18))
sns.heatmap(data.corr(),cmap= cmap,annot=True, square=True)

******************** or simply ****************************

sns.heatmap(data.corr(),annot=True,cmap='Greens')

In [ ]:
#Evaluate the target and find out if our data is imbalanced or not by using countplot

cols= ["#C2C4E2","#EED4E5"]
sns.countplot(x= data["RainTomorrow"], palette= cols)

*********************************************************************

## 4.DATA PREPROCESSING

### a) Identify categorical and numerical features.

In [ ]:
cat_cols = [col for col in data.columns if data[col].dtype == 'object']
num_cols  = [col for col in data.columns if data[col].dtype in['int64', 'float64']]

# Get list of categorical variables and numerical variable

o = (data.dtypes == "object")
object_cols = list(o[o].index)

n = (data.dtypes == "float64")
num_cols = list(n[n].index)

print("Categorical variables:")
print(object_cols)

print("Numeric variables:")
print(num_cols)

***********************      or       **********************

num_attr = data.select_dtypes(['int64','float64']).columns
num_attr

cat_attr = data.select_dtypes('category').columns
cat_attr

In [ ]:
### check for cardinality(number of levels) in categorical variables
for i in cat_attr:
    print('**********************************************')
    print('{0} contains {1} unique values'.format(i, data[i].nunique()))
    print('\n')
    print(data[i].value_counts())

for var in categorical:
    print(var, ' contains ', len(df[var].unique()), ' labels')

In [ ]:
### Check for the frequency Distribution of the target variable
df.target.value_counts() # in numbers
df.target.value_counts(normalize=True) # in percentage

### b) detecting and removing the outliers

In [ ]:
#Detecting outliers
#looking at the scaled features
colours = ["#D0DBEE", "#C2C4E2", "#EED4E5", "#D1E6DC", "#BDE2E2"]
plt.figure(figsize=(20,10))
sns.boxenplot(data = features,palette = colours)
plt.xticks(rotation=90)
plt.show()

In [ ]:
Q1=np.quantile(data['column'],0.25)
Q3=np.quantile(data['column'],0.75)
IQR=Q3-Q1

lower=Q1+1.5*IQR
upper=Q3-1.5*IQR

data=data.loc[(data['column']>lower) & (data['column']<upper),]

### c) data wrangling or data cleaning.
		Filling missing values(imputation)

### Imputation

************************************************

###### ---->For categorical attributes:-



###### ----> Filling missing values  using 'for loop'

In [ ]:
# Filling missing values in categorical variables:

data.isnull().sum()
 
for i in cat_attr:
    data[i].fillna(data[i].mode()[0], inplace=True)
    

# Filling missing values for numerical attrbutes by median
for i in num_attr:
    data[i].fillna(data[i].median()[0], inplace=True)
    
******************* or ******************************


for df1 in [X_train, X_test]:
    for col in numerical:
        col_median=X_train[col].median()
        df1[col].fillna(col_median, inplace=True)    

###### ---->Simple Imputation

In [ ]:
###### for categorical attributes
from sklearn.impute import SimpleImputer

imputer_cat= SimpleImputer(strategy='most_frequent')

imputer_cat = imputer_cat.fit(X_train[cat_attr])

X_train[cat_attr] = imputer_cat.transform(X_train[cat_attr])
X_test[cat_attr] = imputer_cat.transform(X_test[cat_attr])

********************* or *********************
imputer_cat= SimpleImputer(strategy='most_frequent')

imputer_cat = imputer_cat.fit_transform(data[cat_attr])
# impute missing values in X_train and X_test with respective column median in X_train

Imputing missing values with median
imputer = SimpleImputer(strategy='median')
imputer = imputer.fit(X_train[num_attr])

X_train[num_attr] = imputer.transform(X_train[num_attr])
X_test[num_attr] = imputer.transform(X_test[num_attr])


###### ---->TimeSeries imputation by interpolation

In [ ]:
data['Price'] = data['Price'].interpolate(method='linear',limit_direction='forward')

In [ ]:
**********************time series preprocessing**************************************

#Aggregation of a variable containing duplicates with any arithematic operations

agg_data = pd.DataFrame(data.groupby(['Year','Month'],as_index=False)['Price'].mean())

#setting a variable as index 

data.set_index('Date',inplace=True)

***********************************************************

In [ ]:
# customers.groupby('Gender').agg(['mean','min','max', 'std'])['Yearly Amount Spent']

## 5.MODEL BUILDING

###### a)Assining X and y the status of attributes and target

In [ ]:
X = features.drop(["y"], axis=1)
y = features["y"]

In [ ]:
### Rename the target labels(np.where)
#creating bins of numerical variable
For <=50K replace them with 0 and for >50K replace them with 1.

df_train.target = np.where(df_train['target']== '<=50K', 0, 1)
df_test.target =  np.where(df_test['target']== '<=50K.', 0, 1)

**************************** or *******************************

df['y'] = df['y'].apply(lambda x: 0 if x.strip()=='no' else 1)

###### b) Creating dummies (or) perform label, onehotencoding for categorical attribute.
###### ----> Lable encoding -----> If the column is ordinal eg: small / medium / large ==> 0 / 1 / 2

In [ ]:
# Apply label encoder to each column with categorical data
label_encoder = LabelEncoder()
for i in object_cols:
    data[i] = label_encoder.fit_transform(data[i])
    
data.info()

###### ----> One hot encoding -----> If there is no order in the data eg: state (Hyd/ Blr / Chn)-> Hyd(0/1) -> Blr (0/1) -> chn (0/1)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)
X_train_cat = pd.DataFrame(ohe.fit_transform(X_train[cat_cols]), columns = ohe.get_feature_names())
X_test_cat = pd.DataFrame(ohe.transform(X_test[cat_cols]), columns = ohe.get_feature_names())

###### c)Splitting test and training sets

In [ ]:
###train-test split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20,stratify=y,random_state=123)

#when there is a class imbalance in the data the distribution of the class-imbalance will be 
#maintained in the Y_train ,Y_test split also.so that the training of the data will become more accurate

In [33]:
np.mean(abs(scores))

0.9338154770615466

###### d)Perform the scaling of the features

In [ ]:
#standard scaler for the features
col_names=list(num_var)
s_scaler = preprocessing.StandardScaler()
data[num_attr] = s_scaler.fit_transform(data[num_var])

# features = pd.DataFrame(features, columns=col_names) 

# features.describe().T

###### e)Train the Model

In [29]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
cv = KFold(n_splits=5, random_state=1, shuffle=True)

model = LinearRegression()

scores = cross_val_score(model, X, y, cv=cv)

scores

****************************************************

## 6.Evaluate the model

In [ ]:
# Predicting the test set results
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5)

In [ ]:
# confusion matrix
cmap1 = sns.diverging_palette(260,-10,s=50, l=75, n=5, as_cmap=True)
plt.subplots(figsize=(12,8))
cf_matrix = confusion_matrix(y_test, y_pred)
sns.heatmap(cf_matrix/np.sum(cf_matrix), cmap = cmap1, annot = True, annot_kws = {'size':15})

In [ ]:
## Regression assumption plots

def get_plots(y_train,preditctions=results.fittedvalues):
        f = plt.figure(figsize=(20,10))
        ax = f.add_subplot(231)
        sns.scatterplot(y_train,model.fittedvalues,ax=ax,color='r')
        ax.set_title('Check for Linearity:\n Actual Vs Predicted value')
        
        std_residuals = stats.zscore(y_train-results.fittedvalues)
        f = plt.figure(figsize=(14,5))
        ax = f.add_subplot(232)
        sns.scatterplot(model.fittedvalues,std_residuals,ax=ax,color='g')
        ax.set_title('Check for Linearity:\n Predicted value Vs Std.Residuals')

# Check for Residual normality & mean
        ax = f.add_subplot(233)
        sns.distplot((y_train - model.fittedvalues),ax=ax,color='b')
        ax.axvline((y_train - model.fittedvalues).mean(),color='k',linestyle='--')
        ax.set_title('Check for Residual normality & mean: \n Residual eror');

#Residuals - QQ Plot
        sm.qqplot(model.resid, line='s')

        plt.show()

In [ ]:
from scipy import stats


def residual_plots(results):
    import matplotlib.pyplot as plt
    import statsmodels.formula.api as smf
    from statsmodels.nonparametric.smoothers_lowess import lowess
    from scipy import stats
    import statsmodels.api as sm
    
    
    fig, ax = plt.subplots(2,2,figsize=(14,10))
    ########## Residuals vs fitted  ##########
    residuals = results.resid
    fitted = results.fittedvalues
    smoothed = lowess(residuals,fitted)
    top3 = abs(residuals).sort_values(ascending = False)[:3]

    plt.rcParams.update({'font.size': 16})
    plt.rcParams["figure.figsize"] = (8,7)
    
    ax[0,0].scatter(fitted, residuals, edgecolors = 'k', facecolors = 'none')
    ax[0,0].plot(smoothed[:,0],smoothed[:,1],color = 'r')
    ax[0,0].set_ylabel('Residuals')
    ax[0,0].set_xlabel('Fitted Values')
    ax[0,0].set_title('Residuals vs. Fitted')
    ax[0,0].plot([min(fitted),max(fitted)],[0,0],color = 'k',linestyle = ':', alpha = .3)

    for i in top3.index:
        ax[0,0].annotate(i,xy=(fitted[i],residuals[i]))

    #plt.show()
    
    
    ######### Normal qq plot ###########
    sorted_student_residuals = pd.Series(results.get_influence().resid_studentized_internal)
    sorted_student_residuals.index = results.resid.index
    sorted_student_residuals = sorted_student_residuals.sort_values(ascending = True)
    df = pd.DataFrame(sorted_student_residuals)
    df.columns = ['sorted_student_residuals']
    df['theoretical_quantiles'] = stats.probplot(df['sorted_student_residuals'], dist = 'norm', fit = False)[0]
    rankings = abs(df['sorted_student_residuals']).sort_values(ascending = False)
    top3 = rankings[:3]

    #fig, ax = plt.subplots()
    x = df['theoretical_quantiles']
    y = df['sorted_student_residuals']
    ax[0,1].scatter(x,y, edgecolor = 'k',facecolor = 'none')
    ax[0,1].set_title('Normal Q-Q')
    ax[0,1].set_ylabel('Standardized Residuals')
    ax[0,1].set_xlabel('Theoretical Quantiles')
    ax[0,1].plot([np.min([x,y]),np.max([x,y])],[np.min([x,y]),np.max([x,y])], color = 'r', ls = '--')
    for val in top3.index:
        ax[0,1].annotate(val,xy=(df['theoretical_quantiles'].loc[val],df['sorted_student_residuals'].loc[val]))
    #plt.show()
    
    
    ######### Homoscedasticity ############
    student_residuals = results.get_influence().resid_studentized_internal
    sqrt_student_residuals = pd.Series(np.sqrt(np.abs(student_residuals)))
    sqrt_student_residuals.index = results.resid.index
    smoothed = lowess(sqrt_student_residuals,fitted)
    top3 = abs(sqrt_student_residuals).sort_values(ascending = False)[:3]

    #fig, ax = plt.subplots()
    ax[1,0].scatter(fitted, sqrt_student_residuals, edgecolors = 'k', facecolors = 'none')
    ax[1,0].plot(smoothed[:,0],smoothed[:,1],color = 'r')
    ax[1,0].set_ylabel('$\sqrt{|Studentized \ Residuals|}$')
    ax[1,0].set_xlabel('Fitted Values')
    ax[1,0].set_title('Scale-Location')
    ax[1,0].set_ylim(0,max(sqrt_student_residuals)+0.1)
    for i in top3.index:
        ax[1,0].annotate(i,xy=(fitted[i],sqrt_student_residuals[i]))
    #plt.show()
    
    
    ######### Cooks distance  ###########
    student_residuals = pd.Series(results.get_influence().resid_studentized_internal)
    student_residuals.index = results.resid.index
    df = pd.DataFrame(student_residuals)
    df.columns = ['student_residuals']
    df['leverage'] = results.get_influence().hat_matrix_diag
    smoothed = lowess(df['student_residuals'],df['leverage'])
    sorted_student_residuals = abs(df['student_residuals']).sort_values(ascending = False)
    top3 = sorted_student_residuals[:3]

    #fig, ax = plt.subplots()
    x = df['leverage']
    y = df['student_residuals']
    xpos = max(x)+max(x)*0.01  
    ax[1,1].scatter(x, y, edgecolors = 'k', facecolors = 'none')
    ax[1,1].plot(smoothed[:,0],smoothed[:,1],color = 'r')
    ax[1,1].set_ylabel('Studentized Residuals')
    ax[1,1].set_xlabel('Leverage')
    ax[1,1].set_title('Residuals vs. Leverage')
    ax[1,1].set_ylim(min(y)-min(y)*0.15,max(y)+max(y)*0.15)
    ax[1,1].set_xlim(-0.01,max(x)+max(x)*0.05)
    plt.tight_layout()
    for val in top3.index:
        ax[1,1].annotate(val,xy=(x.loc[val],y.loc[val]))

    cooksx = np.linspace(min(x), xpos, 50)
    p = len(results.params)
    poscooks1y = np.sqrt((p*(1-cooksx))/cooksx)
    poscooks05y = np.sqrt(0.5*(p*(1-cooksx))/cooksx)
    negcooks1y = -np.sqrt((p*(1-cooksx))/cooksx)
    negcooks05y = -np.sqrt(0.5*(p*(1-cooksx))/cooksx)

    ax[1,1].plot(cooksx,poscooks1y,label = "Cook's Distance", ls = ':', color = 'r')
    ax[1,1].plot(cooksx,poscooks05y, ls = ':', color = 'r')
    ax[1,1].plot(cooksx,negcooks1y, ls = ':', color = 'r')
    ax[1,1].plot(cooksx,negcooks05y, ls = ':', color = 'r')
    ax[1,1].plot([0,0],ax[1,1].get_ylim(), ls=":", alpha = .3, color = 'k')
    ax[1,1].plot(ax[1,1].get_xlim(), [0,0], ls=":", alpha = .3, color = 'k')
    ax[1,1].annotate('1.0', xy = (xpos, poscooks1y[-1]), color = 'r')
    ax[1,1].annotate('0.5', xy = (xpos, poscooks05y[-1]), color = 'r')
    ax[1,1].annotate('1.0', xy = (xpos, negcooks1y[-1]), color = 'r')
    ax[1,1].annotate('0.5', xy = (xpos, negcooks05y[-1]), color = 'r')
    ax[1,1].legend()
    plt.show()

In [ ]:
from sklearn import metrics
import numpy as np

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

scores = pd.DataFrame(columns=['Model','MAE_Train','MSE_Train','RMSE_Train','MAPE_Train','MAE_Test','MSE_Test','RMSE_Test','MAPE_Test'])

def get_metrics(train_act,train_pred,test_act,test_pred,model_description,dataframe):
    MAE_Train = metrics.mean_absolute_error(train_act,train_pred)
    MSE_Train = metrics.mean_squared_error(train_act,train_pred)
    RMSE_Train = np.sqrt(metrics.mean_squared_error(train_act,train_pred))
    MAPE_Train = mean_absolute_percentage_error(train_act,train_pred)
    MAE_Test = metrics.mean_absolute_error(test_act,test_pred)
    MSE_Test = metrics.mean_squared_error(test_act,test_pred)
    RMSE_Test = np.sqrt(metrics.mean_squared_error(test_act,test_pred))
    MAPE_Test = mean_absolute_percentage_error(test_act,test_pred)
    dataframe = dataframe.append(pd.Series([model_description,MAE_Train,MSE_Train,RMSE_Train,MAPE_Train,MAE_Test,MSE_Test,RMSE_Test,MAPE_Test],
                                           index=scores.columns ), ignore_index=True)
    return(dataframe)

In [ ]:
scores = get_metrics(y_train,model.predict(X_reg),y_test,model.predict(X_test_reg),"Mult_reg",scores)
scores

In [ ]:
### Classification

train_preds = logistic_model.predict(X_train)
train_preds_prob=logistic_model.predict_proba(X_train)[:,1]
test_preds = logistic_model.predict(X_test)
test_preds_prob=logistic_model.predict_proba(X_test)[:,1]

confusion_matrix(y_train,train_preds)

train_accuracy_1= accuracy_score(y_train,train_preds)
train_recall_1= recall_score(y_train,train_preds)
train_precision_1= precision_score(y_train,train_preds)

test_accuracy_1= accuracy_score(y_test,test_preds)
test_recall_1= recall_score(y_test,test_preds)
test_precision_1= precision_score(y_test,test_preds)


https://towardsdatascience.com/top-10-model-evaluation-metrics-for-classification-ml-models-a0a0f1d51b9

threshold = 0.5
train_preds_prob = logit_model.predict(X_train)
train_preds = np.where(train_preds_prob>threshold,1,0)

test_preds_prob = logit_model.predict(X_test)
test_preds = np.where(test_preds_prob>threshold,1,0)

###### R-style logitmodel will predict the probability values and from those probabilities we will compute the classes , but in sklearn LogisticRegression  model.predict gives directly the classes not the probabilities

from sklearn import metrics
import numpy as np
scores = pd.DataFrame(columns=['Model','Train_Accuracy','Train_Recall','Train_Precision','Train_F1_Score','Test_Accuracy','Test_Recall','Test_Precision','Test_F1_Score'])

def get_metrics(train_actual,train_predicted,test_actual,test_predicted,model_description,dataframe):
    train_accuracy = accuracy_score(train_actual,train_predicted)
    train_recall   = recall_score(train_actual,train_predicted,average="weighted")
    train_precision= precision_score(train_actual,train_predicted,average="weighted")
    train_f1score  = f1_score(train_actual,train_predicted,average="weighted")
    test_accuracy = accuracy_score(test_actual,test_predicted)
    test_recall   = recall_score(test_actual,test_predicted,average="weighted")
    test_precision= precision_score(test_actual,test_predicted,average="weighted")
    test_f1score  = f1_score(test_actual,test_predicted,average="weighted")
    dataframe = dataframe.append(pd.Series([model_description, train_accuracy,train_recall,train_precision,train_f1score,
                                            test_accuracy,test_recall,test_precision,test_f1score],
                                           index=scores.columns ), ignore_index=True)
    return(dataframe)

In [ ]:
## ROC And AUC

fpr, tpr, threshold = roc_curve(y_train, train_preds_prob)
roc_auc = auc(fpr, tpr)

%matplotlib notebook
# plt.figure()
plt.plot([0,1],[0,1],color='navy', lw=2, linestyle='--')
plt.plot(fpr,tpr,color='orange', lw=3, label='ROC curve (area = %0.2f)' % roc_auc)

plt.xlabel('FPR')
plt.ylabel('TPR')
plt.legend(loc="lower right")
plt.show()

#### Manual inspection of threshold value

roc_df = pd.DataFrame({'FPR':fpr, 'TPR':tpr, 'Threshold':threshold})
roc_df

roc_df.sort_values('TPR',ascending=False,inplace=True)

optimal_idx = np.argmax(tpr - fpr)
optimal_threshold = threshold[optimal_idx]

optimal_threshold

custom_threshold = 0.099

## To get in 0-1 format vector (pandas Series)
final_pred_array = pd.Series([0 if x>custom_threshold else 1 for x in train_preds_prob])
final_pred_array.value_counts()

final_test_pred_array = pd.Series([0 if x>custom_threshold else 1 for x in test_preds_prob])
final_test_pred_array.value_counts()

## To get True-False format vector (pandas Series)
final_pred = pd.Series(train_preds_prob > 0.099)
print(final_pred.value_counts())
final_test_pred=pd.Series(test_preds_prob > 0.099)
print(final_test_pred.value_counts())


****************************************************

###### how to know that the model is good or not by looking at the errors
        
     ---> we check for mean and sd of the test dataset
         (test.mean(),np.sqrt(test.var()) and compare the how far is the error from mean
            
        

## 7.Hyperparameter tuning and trying with different models

In [ ]:
### 6)GridSearch - Hyper parameter Tuning

from sklearn.model_selection import GridSearchCV
param_grid = {
    'criterion' : ['gini', 'entropy'],
    'max_depth' : [2,5,7,10,12,15,20,24,27, 120, 100],
    'min_samples_leaf': [1, 5, 10],
    'max_leaf_nodes' : [5,10,12,20],
    #'ccp_alpha' : [0.1,0.01,0.001,0.0001],
    'class_weight' : ['balanced','None']
}
dt_gs = GridSearchCV(estimator= DecisionTreeClassifier(), 
                     param_grid= param_grid,
                     cv = 3)
dt_gs.fit(X_train, y_train)
dt_gs.best_params_
best_model = dt_gs.best_estimator_

****************************************************

    
## 8.Prediction or Inference.

In [ ]:
#  Decision Tree visualization

from matplotlib import pyplot as plt
from sklearn import tree

fig = plt.figure(figsize=(20,15))
_ = tree.plot_tree(best_model, 
                   feature_names=X_train.columns,
                   class_names=['<=50','>50'],
                   filled=True)